## Minimum Birkhoff Decomposition: CPLEX Benchmark

This notebook contains a short script to run the CPLEX experiments for the Minimum Birkhoff Decomposition problem: https://arxiv.org/abs/2504.03832 (Sec. 4.3.4).

Select $n \in \{3,4,5,\dots,16\}$ and $\text{type} \in \{\text{sparse}, \text{dense}\}$. 

The maximum runtime per problem instance is limited to 3600 seconds. 



In [ ]:
import numpy as np, pandas as pd
import json, time
from pathlib import Path
import sys
from tqdm.notebook import tqdm

sys.path.append("src")
import model

# Setup 
n = 4                                       # size of the doubly stochastic matrix
type = "sparse"                             # sparse or dense
PATH_TO_QOBLIB = "/Users/victor/Desktop/"   # QOBLIB folder: https://git.zib.de/qopt/qoblib-quantum-optimization-benchmarking-library
LOGS_FOLDER = 'logs'                        # output folder

# Load data
file_name = f'{PATH_TO_QOBLIB}/qoblib-quantum-optimization-benchmarking-library/03-birkhoff/instances/qbench_' + str(n) + '_' + type + '.json'
data = json.load(open(file_name))

# Output dataframe 
df = pd.DataFrame(columns = ["instance", "decomposition_length", "decomposition_error", "time"])

for i in tqdm(range(1, 11)):
    
    x_star = np.array(data[str(i)]['scaled_doubly_stochastic_matrix'])
    Birk = model.BirkCplex(x_star)
    
    start_time = time.time()
    status, permutations, weights = Birk.solve()
    
    if(status=='success'):
        algorithm_runtime = time.time() - start_time
        perm_list = np.reshape(permutations,(len(weights),n,n)) 
        recovered_matrix = np.zeros((n,n))
        for m in range(0,len(perm_list)):
            recovered_matrix = recovered_matrix + weights[m] * perm_list[m] 

        recovered_matrix = recovered_matrix / sum(weights)

        error  = recovered_matrix - np.reshape(x_star, (n,n)) / sum(weights)
        norm_error = np.linalg.norm(error, ord='fro')
        
        df.loc[i] = [str(i), len(weights), norm_error, algorithm_runtime]

Path(f'{LOGS_FOLDER}').mkdir(parents=True, exist_ok=True)
df.to_csv(f'{LOGS_FOLDER}/' + str(n) + '_' + type + '.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/qoblib//qoblib-quantum-optimization-benchmarking-library/03-birkhoff/instances/qbench_4_sparse.json'